In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
from scipy.integrate import odeint
from scipy.integrate import quad
import seaborn as sns
from matplotlib import rcParams
from scipy.interpolate import InterpolatedUnivariateSpline
plt.rcParams.update({
  "text.usetex": True,
  "font.family": "serif"
})




π = np.pi
G = 4.4959e-15            #in units of M☉^-1 pc^3 yr^-2
c = 0.3068                #in units of pc yr^-1
ρ_eq = 3.1812e3           #in units of M☉ pc^-3 with ρ_eq=2.15368e-16 kg m^-3
t = 13.78e9               #in units of yrs corresponding to t_0=13.78Gyr
yr = 3.154e7              # in units of seconds
pc = 3.0857e16            # in meters
yr = 3.154e7              # in units of seconds
t_eq = 1.59246e12/yr      # in units of seconds
σ_eq = 0.005
ρ_m  = 4e19                #ρ_m=4e19 M☉ Gpc^-3


t_m = 13.78e9             #in units of yr 









m_1 = np.geomspace(1e-3, 100, 10)
m_2 = np.geomspace(1e-3, 100, 10)

m_1_list = m_1.tolist()
m_2_list = m_2.tolist()


print("m_1 = ", m_1_list)

Δ_1_list = np.zeros(len(m_1_list))
Δ_2_list = np.zeros(len(m_2_list))



for i, (m_1, m_2)  in enumerate(zip(m_1_list, m_2_list)):
    y = np.log(m_1)
    dy =  np.log(m_1) - np.log(0.5 * m_2)
    Δ_1_list[i] =  Δ_2_list[i] = np.exp(y + 0.5*dy) - np.exp(y - 0.5*dy)
    




# Formatting x and y-axis labels.

def sciformat_3(x):
    n = np.floor(np.log10(x))
    m = x*(10.0**(-n))
    str1 = ""
    if (n in [0, 1]):
        return r'$%d$'%(x,)
    elif (m > 1.001):
        str1 = r'$%d \times ' % (m,)
        str2 = r'10^{%d}$' % n
    else:
        str2 = r'$10^{%d}$' % n
    return str1 + str2

xlabel_list = np.zeros(len(m_1_list), dtype ='U100')
ylabel_list = np.zeros(len(m_2_list), dtype ='U100')

for i, x in enumerate(m_1_list):
    xlabel_list[i] = sciformat_3(x)
    
for i, x in enumerate(m_2_list):
    ylabel_list[i] = sciformat_3(x)

    
    
m_1 = 1       #in units of solar mass
m_2 = 1e-3    # in units of solar mass

Δ_1 = Δ_1_list[0]
Δ_2 = Δ_2_list[1]



m_1 =  [0.001, 0.003593813663804626, 0.01291549665014884, 0.046415888336127795, 0.1668100537200059, 0.5994842503189409, 2.1544346900318843, 7.742636826811277, 27.825594022071257, 100.0]


In [2]:
    
# Merger Rate calculations for IMRIPY systems assuming that the DM halos are intact in the merging process.
file = np.load('tmofjo.npz')
ji_ref_list = file['arr_0']
tm_ref_list = file['arr_1']

print("ji_ref = ", ji_ref_list)
print("tm_ref = ", tm_ref_list, "yr")
  
g =  InterpolatedUnivariateSpline(tm_ref_list, ji_ref_list, ext = 2) # j_i = g(t_m)
g_prime = g.derivative()




def Merger_imripy(m_1, m_2, Δ_1, Δ_2, t_m):
    
    if m_2 > m_1:
        return 0
        
    else:    
    
        beta = 0.75
        gamma  =  0.65
        delta = -0.89
        a_i_ref = (1e4 * 0.04606)   # in units of pc
        m1_ref = 1      # in units of solar mass.
        m2_ref = 1e-3   # in units of solar mass.

        def f_(m):
            return f*P(m)

        def f_b(m_1,m_2):
            return  f_(m_1)+f_(m_2)

        def λ(m_1, m_2, Δ_1, Δ_2, X): # λ_0
            return  X/(f_b(m_1,m_2)*np.sqrt(Δ_1*Δ_2))


        def s(m_1, m_2, Δ_1, Δ_2, X): # S_dec of the binary decouple.
            return ((0.03095*λ(m_1, m_2, Δ_1, Δ_2, X)+ 0.3425)*λ(m_1, m_2, Δ_1, Δ_2, X))

        def t(m_1, m_2, Δ_1, Δ_2, X):  #t_dec of the binary decouple.
            return ((np.sqrt(3/(8*π*G*ρ_eq)))*(((2/3)*(s(m_1, m_2, Δ_1, Δ_2, X)-2)*np.sqrt(s(m_1, m_2, Δ_1, Δ_2, X)+1))+(4/3)))


        def ρ_bar(m):
            return ((ρ_eq/2)*(t_eq**(3/2))*((2*G*m)**(3/4))) 

        def M_1halo(m_1, m_2, Δ_1, Δ_2, X):
            return (((16*π)/3)*ρ_bar(m_1)*((2*G*m_1*(t(m_1, m_2, Δ_1, Δ_2, X)**2))**(1/4)))


        def M_2halo(m_1, m_2, Δ_1, Δ_2, X):
            return (((16*π)/3)*ρ_bar(m_2)*((2*G*m_2*(t(m_1, m_2, Δ_1, Δ_2, X)**2))**(1/4)))


        def M_binary_initial(m_1, m_2, Δ_1, Δ_2, X):
            return (m_1 + m_2 + M_1halo(m_1, m_2, Δ_1, Δ_2, X) + M_2halo(m_1, m_2, Δ_1, Δ_2, X))


        def D_halo(m_1, m_2, Δ_1, Δ_2, X):
            return ( 1 + 0.0857 * λ(m_1, m_2, Δ_1, Δ_2, X))/ np.sqrt(1 + 0.0583 * λ(m_1, m_2, Δ_1, Δ_2, X))

        X = np.geomspace(1e-3, 1e5, 1000000)

        def j_X(m_1, m_2, Δ_1, Δ_2, X):
            return 0.4087 * np.sqrt(f**2 + 0.2998 * (σ_eq**2)) * D_halo(m_1, m_2, Δ_1, Δ_2, X) * (X/(f_b(m_1,m_2) * np.sqrt(Δ_1*Δ_2)))


        def PDF(m):
            return (P(m)/m)

        I = quad(PDF, 1e-4, np.inf)         # Taking limits of the PBH mass from 5 M☉ to 100 M☉ instead of 
        # 0 to infinity.

        n_T = f*ρ_eq*I[0]                  #n_T=f*ρ_eq∫(P(m)/m)*dm 

        def x̄(m_1,m_2,Δ_1,Δ_2):
            return (((3*(m_1 + m_2))/(8*π*ρ_eq*f_b(m_1,m_2)*np.sqrt(Δ_1*Δ_2)))**(1/3))


        def a_initial(m_1, m_2, Δ_1, Δ_2, X):  # for x = x_bar 
            return (0.0977*λ(m_1,m_2,Δ_1,Δ_2, X) + 0.0057*(λ(m_1,m_2,Δ_1,Δ_2, X)**2)) * x̄(m_1,m_2,Δ_1,Δ_2) * (X**(1/3))

        print("a_i =", a_initial(m_1, m_2, Δ_1, Δ_2, X))
        
        def C(m_1,m_2,Δ_1,Δ_2, X):
            return ((a_initial(m_1,m_2,Δ_1,Δ_2, X)/a_i_ref)**(beta)) * ((m_1/ m1_ref)**(gamma)) * ((m_2/m2_ref)**(delta))


        print("tm/C =",(t_m/C(m_1,m_2,Δ_1,Δ_2, X)) )


        def j_i(m_1,m_2,Δ_1,Δ_2, X):
            return  g(t_m/C(m_1,m_2,Δ_1,Δ_2, X))


        def j_i_prime(m_1,m_2,Δ_1,Δ_2, X):
            return (1/C(m_1,m_2,Δ_1,Δ_2, X)) * g_prime(t_m/C(m_1,m_2,Δ_1,Δ_2, X)) 


        def γ_X(m_1,m_2,Δ_1,Δ_2, X):    
            return j_i(m_1,m_2,Δ_1,Δ_2, X)/ j_X(m_1, m_2, Δ_1, Δ_2, X)



        def integrand(m_1,m_2,Δ_1,Δ_2, X):
            return j_i_prime(m_1,m_2,Δ_1,Δ_2, X) * (1/j_i(m_1,m_2,Δ_1,Δ_2, X)) * ((γ_X(m_1,m_2,Δ_1,Δ_2, X)**2)/ ((1 + (γ_X(m_1,m_2,Δ_1,Δ_2, X)**2))**(3/2))) * np.exp(- X * ((4*π)/3) * (x̄(m_1,m_2,Δ_1,Δ_2)**3) * n_T)



        return  (ρ_m/2) * (((f**2)* (m_1 + m_2))/(f_b(m_1,m_2) * np.sqrt(Δ_1*Δ_2))) * np.minimum((P(m_1)*Δ_1)/m_1, (P(m_2)*Δ_2)/m_2) * ((P(m_1)*Δ_1)/m_1 + (P(m_2)*Δ_2)/m_2) *   (np.trapz(integrand(m_1,m_2,Δ_1,Δ_2, X),X))

ji_ref =  [0.001      0.00215443 0.00464159 0.01       0.02154435 0.04641589
 0.1        0.21544347 0.46415888 1.        ]
tm_ref =  [8.26932584e+07 1.17408713e+08 1.61142613e+08 2.08429710e+08
 2.47907838e+08 2.71268360e+08 2.82836183e+08 3.06696976e+08
 4.66662899e+08 1.75524234e+09] yr


In [3]:
 # Calculation of the 2D merger rate for Powerlaw PBH mass distribution. 
# The resolution of the PBHs masses m_1 is Δ_1 and of m_2 is Δ_2.


Merger_imripy_PWR = np.zeros((10, 10))



for i in range(10):
    for j in range(10):
        
        f_pbh = 4.3e-3
        f = 0.85*f_pbh
        
          
        def P(m): #Powerlaw PBH mass distribution
            α = 3.6
            M = 1e-4    #in units of M☉
            return ((α-1)/M) * ((m/M)**(-α))

        if i==j:
            Merger_imripy_PWR[i,j] = 0.5 * Merger_imripy(m_1, m_2, Δ_1, Δ_2, t_m)
            
        else:
            Merger_imripy_PWR[i,j] = Merger_imripy(m_1, m_2, Δ_1, Δ_2, t_m)   
    
        



# Plot  for IMRIPY Merger rate using Power Law PBH mass function.

plt.rcParams['font.family'] = 'serif'
plt.rcParams['xtick.direction'] = 'out'
plt.rcParams['ytick.direction'] = 'out'



Merger_imripy_PWR = np.load('Merger_imripy_PWR.npy')

mask = np.tril(np.ones((10,10)), k=-1)  #created an array of shape Merger_12 with all the elements in it being 0.
fig,ax = plt.subplots(figsize=(10, 7))  



ax = sns.heatmap(Merger_imripy_PWR, mask = mask,  cmap = 'viridis', cbar = True, 
                 cbar_kws = {"shrink": 1,'label': '$Merger \: Rate, \: \mathcal{R}_{ij} \: [\mathrm{Gpc}^{-3} \, \mathrm{yr}^{-1}]$'})

    

# make frame visible
for _, spine in ax.spines.items():
    spine.set_visible(True)
    spine.set_linewidth(1)
    



ax.set_xticklabels(xlabel_list)
ax.set_yticklabels(ylabel_list,rotation = 360)





ax.invert_yaxis()
plt.xlabel('$m_{i}\:[M_{\odot}]$', fontsize = 18)
plt.ylabel('$m_{j}\: [M_{\odot}]$', fontsize = 18)
plt.title('Power Law PDF for IMRIPY systems')
plt.savefig('IMRIPYPWRLAW.pdf', bbox_inches = 'tight')
plt.show()     

a_i = [9.09342609e-01 9.09375761e-01 9.09408915e-01 ... 2.72557102e+18
 2.72568817e+18 2.72580533e+18]
tm/C = [1.47128400e+12 1.47124377e+12 1.47120354e+12 ... 2.04243875e-02
 2.04237291e-02 2.04230707e-02]


ValueError: Found x value not in the domain